# HHEM demo with Gradio-client

The [Gradio Python client](https://www.gradio.app/guides/getting-started-with-the-python-client) makes it very easy to use any Gradio app as an API.
In this notebook we demonstrate how to use HHEM - [Vectara](https://vectara.com)'s [Hallucination Evaluation Model](https://huggingface.co/vectara/hallucination_evaluation_model) - with the gradio python client.


In [1]:
from gradio_client import Client
import os

In [2]:
# Initialize the client and point it to HHEM space

hf_token = os.environ['HF_TOKEN']
client = Client("vectara/HHEM", hf_token=hf_token)

Loaded as API: https://vectara-hhem.hf.space ✔


We can inspect what endpoints are exposed in the client via the `view_api()` call:

In [3]:
client.view_api()

Client.predict() Usage Info
---------------------------
Named API endpoints: 1

 - predict(sentence1, sentence2, api_name="/predict") -> hhem_score
    Parameters:
     - [Textbox] sentence1: str (required)  
     - [Textbox] sentence2: str (required)  
    Returns:
     - [Label] hhem_score: Dict(label: str | int | float | None, confidences: List[Dict(label: str | int | float | None, confidence: float | None)] | None) 



We can see that the "/predict" endpoint of this space accepts two sentences and return a hallucination evaluation score.

Let's try this out for a few examples, to demonstrate the hallucination detection capabilities.
When calling HHEM, we use two sentences
- The first sentence includes the context - a set of facts to summarize
- The second sentence includes the proposed summary

HHEM scores the factual consistency of the 2nd sentence against the facts in the 1st sentence and produces a score of 0...1, 1 being most factually consistent and 0 means not at all.


In [4]:
test_pairs = [
    [
        "Goldfish are being caught weighing up to 2kg and koi carp up to 8kg and one metre in length", 
        "Koi carp can be as heavy as 2kg and as long as one meter"],
    [
        "The first vaccine for Ebola was approved by the FDA in 2019 in the US, five years after the initial outbreak in 2014. To produce the vaccine,  scientists had to sequence the DNA of Ebola, then identify possible vaccines, and finally show successful clinical trials.  Scientists say a vaccine for COVID-19 is unlikely to be ready this year, although clinical trials have already started.", 
        "COVID-19 vaccine will be available this year."
    ], 
    ['Water is tasteless', 'You can not taste water.'], 
    ['I am happy.', 'I am pleasant.'],
    ["Mark Wahlberg was a fan of Manny.", "Manny was a fan of Mark Wahlberg."], 
    ['I went to the United States.', 'I visited California.'],
    ['I visited California.', 'I went to the United States.'],
    [
        "`` Hourglass'' is a song by British electronic duo Disclosure . Hourglass is singer-songwriter James Taylor 's fourteenth studio album .", 
        'The passage mentions two different "Hourglass" entities: one is a song by the British electronic duo Disclosure, and the other is the fourteenth studio album by singer-songwriter James Taylor.'
    ],
    [
        "Gradio-client is a library from Huggingface. You can use it to make any Huggingface space immediately available for prediction", 
        "Gradio-client is a new library that enables developers to use Huggingface spaces as prediction endpoints"
    ]
]

In [5]:
for pair in test_pairs:
    print(f"Sentence 1: {pair[0]}")
    print(f"Sentence 2: {pair[1]}")
    print(f"HHEM score = {client.predict(pair[0], pair[1])['label']}")
    print("\n")

Sentence 1: Goldfish are being caught weighing up to 2kg and koi carp up to 8kg and one metre in length
Sentence 2: Koi carp can be as heavy as 2kg and as long as one meter
HHEM score = 0.0053


Sentence 1: The first vaccine for Ebola was approved by the FDA in 2019 in the US, five years after the initial outbreak in 2014. To produce the vaccine,  scientists had to sequence the DNA of Ebola, then identify possible vaccines, and finally show successful clinical trials.  Scientists say a vaccine for COVID-19 is unlikely to be ready this year, although clinical trials have already started.
Sentence 2: COVID-19 vaccine will be available this year.
HHEM score = 0.0003


Sentence 1: Water is tasteless
Sentence 2: You can not taste water.
HHEM score = 0.9364


Sentence 1: I am happy.
Sentence 2: I am pleasant.
HHEM score = 0.5507


Sentence 1: Mark Wahlberg was a fan of Manny.
Sentence 2: Manny was a fan of Mark Wahlberg.
HHEM score = 0.0028


Sentence 1: I went to the United States.
Sentence